In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

import time

apr2014 = date(2014,1,4)
sept2014 = date(2014,9,30)
sept2015 = date(2015,9,30)
sept2016 = date(2016,9,30)
sept2017 = date(2017,9,30)
sept2018 = date(2018,9,30)



In [2]:
transactions = pd.read_csv('./CleanData/transactions.csv',low_memory=False)
constituents = pd.read_csv('./constituents_with_target.csv',low_memory=False)
donorInteractions = pd.read_csv('./CleanData/donorInteractions.csv',low_memory=False)
mailActions = pd.read_csv('./CleanData/mailActions.csv',low_memory=False)
pledges = pd.read_csv('./CleanData/pledges.csv',low_memory=False)

In [3]:
#Data splitting into years
transactions['date'] = pd.to_datetime(transactions['date'])

y1transactions = transactions[(transactions['date'].dt.date >= sept2014) & (transactions['date'].dt.date < sept2015)]
y2transactions = transactions[(transactions['date'].dt.date >= sept2015) & (transactions['date'].dt.date < sept2016)]
y3transactions = transactions[(transactions['date'].dt.date >= sept2016) & (transactions['date'].dt.date < sept2017)]

donorInteractions['actionDate'] = pd.to_datetime(donorInteractions['actionDate'])

y1donorInteractions = donorInteractions[(donorInteractions['actionDate'].dt.date >= sept2014) & (donorInteractions['actionDate'].dt.date < sept2015)]
y2donorInteractions = donorInteractions[(donorInteractions['actionDate'].dt.date >= sept2015) & (donorInteractions['actionDate'].dt.date < sept2016)]
y3donorInteractions = donorInteractions[(donorInteractions['actionDate'].dt.date >= sept2016) & (donorInteractions['actionDate'].dt.date < sept2017)]

pledges['date'] = pd.to_datetime(pledges['date'])

y1pledges = pledges[(pledges['date'].dt.date >= sept2014) & (pledges['date'].dt.date < sept2015)]
y2pledges = pledges[(pledges['date'].dt.date >= sept2015) & (pledges['date'].dt.date < sept2016)]
y3pledges = pledges[(pledges['date'].dt.date >= sept2016) & (pledges['date'].dt.date < sept2017)]



In [4]:
#create features for different years

newfeatures = pd.DataFrame()

newfeatures['y1numDonations'] = y1transactions.groupby('cnBioId')['cnBioId'].count()
newfeatures['y2numDonations'] = y2transactions.groupby('cnBioId')['cnBioId'].count()
newfeatures['y3numDonations'] = y3transactions.groupby('cnBioId')['cnBioId'].count()

#newfeatures['firstDonationDate'] = transactions.groupby('cnBioId')['date'].min()
#newfeatures['lastDonationDate'] = transactions.groupby('cnBioId')['date'].max()

newfeatures['y1amountSum'] = y1transactions.groupby('cnBioId')['amount'].sum()
newfeatures['y2amountSum'] = y2transactions.groupby('cnBioId')['amount'].sum()
newfeatures['y3amountSum'] = y3transactions.groupby('cnBioId')['amount'].sum()

newfeatures['y1amountMean'] = y1transactions.groupby('cnBioId')['amount'].mean()
newfeatures['y2amountMean'] = y2transactions.groupby('cnBioId')['amount'].mean()
newfeatures['y3amountMean'] = y2transactions.groupby('cnBioId')['amount'].mean()

newfeatures['y1amountSTD'] = y1transactions.groupby('cnBioId')['amount'].std(ddof=0)
newfeatures['y2amountSTD'] = y2transactions.groupby('cnBioId')['amount'].std(ddof=0)
newfeatures['y3amountSTD'] = y3transactions.groupby('cnBioId')['amount'].std(ddof=0)


newfeatures['y1numInteractions'] = y1donorInteractions.groupby('cnBioId')['cnBioId'].count()
newfeatures['y2numInteractions'] = y2donorInteractions.groupby('cnBioId')['cnBioId'].count()
newfeatures['y3numInteractions'] = y3donorInteractions.groupby('cnBioId')['cnBioId'].count()

#no date information for mailings

newfeatures['numMailings'] = mailActions.groupby('cnBioId')['cnBioId'].count()
newfeatures['numMailings'] = newfeatures['numMailings'].fillna(0)

newfeatures['y1pledgeAmount'] = y1pledges.groupby('cnBioId')['amount'].sum()
newfeatures['y2pledgeAmount'] = y2pledges.groupby('cnBioId')['amount'].sum()
newfeatures['y3pledgeAmount'] = y3pledges.groupby('cnBioId')['amount'].sum()
#newfeatures['firstPledgeDate'] = pledges.groupby('cnBioId')['date'].min()
#newfeatures['firstPledgeDate'] = newfeatures['firstPledgeDate'].fillna(0)
#newfeatures['lastPledgeDate'] = pledges.groupby('cnBioId')['date'].max()
#newfeatures['lastPledgeDate'] = newfeatures['lastPledgeDate'].fillna(0)

newfeatures.update(newfeatures.fillna(0))

newfeatures.head()




,y1numDonations,y2numDonations,y3numDonations,y1amountSum,y2amountSum,y3amountSum,y1amountMean,y2amountMean,y3amountMean,y1amountSTD,y2amountSTD,y3amountSTD,y1numInteractions,y2numInteractions,y3numInteractions,numMailings,y1pledgeAmount,y2pledgeAmount,y3pledgeAmount
cnBioId,,,,,,,,,,,,,,,,,,,
100011,1,2.0,1.0,25.0,50.0,25.0,25.0,25.0,25.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100031,2,1.0,1.0,60.0,40.0,40.0,30.0,40.0,40.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100037,1,0.0,0.0,20.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100045,1,1.0,1.0,500.0,500.0,500.0,500.0,500.0,500.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100078,3,1.0,3.0,150.0,50.0,170.0,50.0,50.0,50.0,0.0,0.0,4.714045,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
print(constituents.shape)
target = pd.merge(left=constituents,right=newfeatures, left_on='cnBioId', right_on='cnBioId',how='left')
   
print(target.shape)



(45528, 57)
(45528, 76)


In [6]:
target.update(target[['y1numDonations','y2numDonations','y3numDonations','y1amountSum','y2amountSum','y3amountSum','y1amountMean','y2amountMean','y3amountMean','y1amountSTD','y2amountSTD','y3amountSTD','y1numInteractions','y2numInteractions','y3numInteractions','numMailings','y1pledgeAmount','y2pledgeAmount','y3pledgeAmount']].fillna(0))


In [7]:
target.head()

,cnBioId,amount_last_year,target_1000,target_10000,age,anonymous,deceasedDate,donorType,noValidAddresses,receiptType,...,y1amountSTD,y2amountSTD,y3amountSTD,y1numInteractions,y2numInteractions,y3numInteractions,numMailings,y1pledgeAmount,y2pledgeAmount,y3pledgeAmount
0,9,0.00,0,0,NaN,0.0,NaN,individual,0.0,One receipt per gift,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,0.00,0,0,NaN,0.0,NaN,individual,0.0,One receipt per gift,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,25.00,0,0,NaN,1.0,NaN,individual,0.0,One receipt per gift,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,275.00,0,0,NaN,0.0,NaN,individual,0.0,Consolidated receipts,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,101,9885.27,1,0,NaN,1.0,NaN,individual,0.0,One receipt per gift,...,489.728772,838.360783,1411.531399,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
target.to_csv('constituents_with_target_features.csv', index=False)